In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

from constrerl.erl_schema import convert_to_output, Article, TernaryMentionBasedRelation, TernaryTagBasedRelation, BinaryTagBasedRelation
from constrerl.annotator import load_test, load_train
import glob
from pathlib import Path
import json
import pandas as pd
from collections.abc import Callable, Awaitable


In [ ]:
set_type = "dev"
results_dir = Path(f"../data/results_{set_type}")
merge_dir = Path(f"../data/merge_self_{set_type}")

onto_file = f"../data/tug_{set_type}_results.json"

results_dir = Path(results_dir)
onto_file = Path(onto_file)

In [5]:
onto_data = load_train(onto_file)
result_datas: dict[str, dict[str, Article]] = {
    rf.name.split(".")[0]: load_train(rf) for rf in results_dir.glob("*.json")
}

In [6]:
for k, result_data in result_datas.items():
    merged_data: dict[str, dict[str, Article]] = {}
    for id, v in onto_data.items():
        tmbrs = v.ternary_mention_based_relations
        tug_result = (
            result_data[id]
            if id in result_data
            else Article(ternary_mention_based_relations=[])
        )
        ttbrs_onto = [
            (ttbr.subject_label, ttbr.predicate, ttbr.object_label) for ttbr in tmbrs
        ]

        ttbrs_tug = [
            (ttbr.subject_label, ttbr.predicate, ttbr.object_label)
            for ttbr in tug_result.ternary_mention_based_relations
        ]
        id_sets = {
            "union": set(ttbrs_tug).union(ttbrs_onto),
            "intersection": set(ttbrs_tug).intersection(ttbrs_onto),
        }
        merged_articles = {
            n: Article(
                ternary_mention_based_relations=[],
                ternary_tag_based_relations=[],
                binary_tag_based_relations=[],
            )
            for n in id_sets.keys()
        }
        for n, id_set in id_sets.items():
            for id_tuple in id_set:
                s, p, o = id_tuple
                tmbrs_combined = [
                    tmbr
                    for tmbr in tmbrs + tug_result.ternary_mention_based_relations
                    if (tmbr.subject_label, tmbr.predicate, tmbr.object_label)
                    == id_tuple
                ]
                for tmbr in tmbrs_combined:
                    merged_articles[n].ternary_mention_based_relations.append(
                        TernaryMentionBasedRelation(
                            subject_label=tmbr.subject_label,
                            predicate=tmbr.predicate,
                            object_label=tmbr.object_label,
                            subject_text_span=tmbr.subject_text_span,
                            object_text_span=tmbr.object_text_span,
                        )
                    )
                merged_articles[n].ternary_tag_based_relations.append(
                    TernaryTagBasedRelation(
                        subject_label=s,
                        predicate=p,
                        object_label=o,
                    )
                )
                merged_articles[n].binary_tag_based_relations.append(
                    BinaryTagBasedRelation(
                        subject_label=s,
                        object_label=o,
                    )
                )
            if n not in merged_data:
                merged_data[n] = {}
            merged_data[n][id] = merged_articles[n]
        # ttbrs = [
        #     TernaryTagBasedRelation.model_validate_json(tmbr.model_dump_json())
        #     for tmbr in tmbrs
        # ]
        # btrs = [
        #     BinaryTagBasedRelation.model_validate_json(tmbr.model_dump_json())
        #     for tmbr in tmbrs
        # ]
    for n, merged_data_method in merged_data.items():
        merged_articles = {
            id: article.model_dump() for id, article in merged_data_method.items()
        }
        with open(merge_dir / f"{k}_{n}.json", "w") as f:
            json.dump(merged_articles, f, indent=4)